In [1]:
seed = 1000
GPU = 1
num_workers = 4

In [2]:
import os
import random
import itertools
from decimal import Decimal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import datetime as dt

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

In [4]:
from sklearn.model_selection import KFold

In [5]:
sns.set(style="whitegrid", palette="muted", color_codes=True, font_scale=1.25)

In [6]:
torch.cuda.set_device(GPU)
print('Available devices:', torch.cuda.device_count())
print('Current cuda device:', torch.cuda.current_device())
print(torch.cuda.get_device_name(GPU))

Available devices: 8
Current cuda device: 1
GeForce RTX 2080 Ti


In [7]:
def seed_everything(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed)

In [8]:
month = '0{}'.format(dt.now().month) if dt.now().month < 10 else str(dt.now().month)
day = '0{}'.format(dt.now().day) if dt.now().day < 10 else str(dt.now().day)
hour = '0{}'.format(dt.now().hour) if dt.now().hour < 10 else str(dt.now().hour)
minute = '0{}'.format(dt.now().minute) if dt.now().minute < 10 else str(dt.now().minute)
sec = '0{}'.format(dt.now().second) if dt.now().second < 10 else str(dt.now().second)

In [9]:
save_path = "/users/hjw/data/ABCD/DNN/"
output_folder = save_path + "{}{}{}_{}{}{}".format(21, month, day, hour, minute, sec)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
print(output_folder)

/users/hjw/data/ABCD/DNN/210404_122020


In [10]:
data = np.load("/users/hjw/data/ABCD/npz_files/MDD_binary_without_NONE.npz", 
               allow_pickle=True)
X = stats.zscore(data["X"], axis=1)
y = data["y"].reshape(-1, 1)
print(X.shape, y.shape)

(1356, 46360) (1356, 1)


In [11]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
train_folds_idx = []
test_folds_idx = []
for train_idx, test_idx in kf.split(X):
    train_folds_idx.append(train_idx)
    test_folds_idx.append(test_idx)

In [12]:
epochs = 500
batch_size = 128
learning_rate = 1e-03
optimizer_name = 'nag'
act_func_name = "prelu"

mode = "max"
patience = 5
factor = 0.25
min_lr = 1e-15

momentum = 0.90
l1_param = None
l2_param = 1e-02
dropout_rate = 0.8

n_classes = 1
input_dim = X.shape[-1]
output_dim = n_classes

outer_n_splits = len(train_folds_idx)
inner_n_splits = outer_n_splits - 1

In [13]:
n_hidden = [128, 128]
beta_lr = [1e-03, 1e-02]
max_beta = [5e-02, 5e-01]
wsc_flag = [1, 1]
num_sparsity_layers = wsc_flag.count(1)
hsp_cand_1 = [0.95, 0.9, 0.8]
hsp_cand_2 = [0.8, 0.5, 0.3]
indices = [i + 1 for i, x in enumerate(wsc_flag) if x == 1]
hsp_cand_list = list(itertools.product(hsp_cand_1, hsp_cand_2))
hsp_cand_list = [list(i) for i in hsp_cand_list]
hsp_cand = [hsp_cand_1, hsp_cand_2]

In [14]:
f = open(output_folder + "/parameters.txt", 'w')
f.write('Optimizer : ' + str(optimizer_name) + '\n')
f.write('Activation function : ' + str(act_func_name) + '\n')
f.write('Epochs : ' + str(epochs) + '\n')
f.write('Batch size : ' + str(batch_size) + '\n')
f.write('Learning rate : ' + str(learning_rate) + '\n')
f.write('Patience : ' + str(patience) + '\n')
f.write('Factor : ' + str(factor) + '\n')
f.write('Minimum LR : ' + str(min_lr) + '\n')
f.write('L2_reg : ' + str(l2_param) + '\n')
f.write('Number of hidden layer : ' + str(len(n_hidden)) + '\n')
f.write('Number of hidden node : ' + str(n_hidden) + '\n')
f.write('HSP candidate : ' + str(hsp_cand) + '\n')
f.write('Beta learning rate : ' + str(beta_lr) + '\n')
f.write('Max beta : ' + str(max_beta) + '\n')
f.write('Dropout rate : ' + str(dropout_rate) + '\n')
f.close()

In [15]:
# Training dataset
class train_dataset(Dataset): 
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, idx): 
        X_train = torch.from_numpy(self.X_train[idx]).type(torch.FloatTensor)
        y_train = torch.from_numpy(self.y_train[idx]).type(torch.FloatTensor)

        return X_train, y_train

In [16]:
# Test dataset
class test_dataset(Dataset): 
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        
    def __len__(self):
        return len(self.X_test)
    
    def __getitem__(self, idx): 
        X_test = torch.from_numpy(self.X_test[idx]).type(torch.FloatTensor)
        y_test = torch.from_numpy(self.y_test[idx]).type(torch.FloatTensor)
        
        return X_test, y_test

In [17]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [18]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.hidden_1 = nn.Linear(input_dim, n_hidden[0])
        self.batch_norm_1 = nn.BatchNorm1d(n_hidden[0])
        self.hidden_2 = nn.Linear(n_hidden[0], n_hidden[1])
        self.batch_norm_2 = nn.BatchNorm1d(n_hidden[1])
        self.output_layer = nn.Linear(n_hidden[1], output_dim)
        self.act_func = get_activation_function(act_func_name)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.weights_init()
    
    def forward(self, x):
        x = self.hidden_1(x)
        x = self.batch_norm_1(x)
        x = self.act_func(x)
        x = self.dropout(x)
        x = self.hidden_2(x)
        x = self.batch_norm_2(x)
        x = self.act_func(x)
        x = self.dropout(x)
        out = self.output_layer(x)
        return out

    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # He initialization for ReLU
                nn.init.kaiming_normal_(m.weight)
                # Bias initialization
                nn.init.normal_(m.bias, std=0.01)

In [19]:
def get_optimizer(model, opt_name):
    lower_opt_name = opt_name.lower()
    if lower_opt_name == 'momentum':
        return optim.SGD(model.parameters(), lr=learning_rate, 
                         momentum=momentum, weight_decay=l2_param)
    elif lower_opt_name == 'nag':
        return optim.SGD(model.parameters(), lr=learning_rate, 
                         momentum=momentum, weight_decay=l2_param, nesterov=True)
    elif lower_opt_name == 'adam':
        return optim.Adam(model.parameters(), lr=learning_rate, 
                          weight_decay=l2_param)
    else:
        sys.exit("Illegal arguement for optimizer type")

In [20]:
def get_activation_function(act_func_name):
    act_func_name = act_func_name.lower()
    if act_func_name == 'relu':
        return nn.ReLU()
    elif act_func_name == 'prelu':
        return nn.PReLU()
    elif act_func_name == 'elu':
        return nn.ELU()
    elif act_func_name == 'silu':
        return nn.SiLU()
    elif act_func_name == 'leakyrelu':
        return nn.LeakyReLU()
    elif act_func_name == 'tanh':
        return nn.Tanh()
    else:
        sys.exit("Illegal arguement for activation function type")

In [21]:
def init_hsp():
    hsp_val = np.zeros(num_sparsity_layers)
    beta_val = hsp_val.copy()
    hsp_list = np.zeros((num_sparsity_layers, epochs))
    beta_list = np.zeros((num_sparsity_layers, epochs))
    
    return hsp_val, beta_val, hsp_list, beta_list

In [22]:
# Weight sparsity control with Hoyer's sparsness (Layer wise)
def calc_hsp(w, beta, max_beta, beta_lr, tg_hsp):
    
    # Get value of weight
    [dim, n_nodes] = w.shape
    num_elements = dim * n_nodes
    norm_ratio = torch.norm(w, 1) / torch.norm(w, 2)

    # Calculate hoyer's sparsity level
    num = np.sqrt(num_elements) - norm_ratio.item()
    den = np.sqrt(num_elements) - 1
    hsp = num / den

    # Update beta
    beta = beta + beta_lr * np.sign(tg_hsp - hsp)
    
    # Trim value
    beta = 0.0 if beta < 0.0 else beta
    beta = max_beta if beta > max_beta else beta

    return [hsp, beta]

In [23]:
def l1_penalty(model, epoch, hsp_val, beta_val, hsp_list, beta_list, tg_hsp):
    l1_reg = None
    l1_val = 0
    layer_idx = 0
    wsc_idx = 0

    for name, param in model.named_parameters():
        if "hidden"  in name and "weight" in name:
            temp_w = param
            if wsc_flag[layer_idx] != 0:
                hsp_val[wsc_idx], beta_val[wsc_idx] = calc_hsp(
                    temp_w, beta_val[wsc_idx], max_beta[wsc_idx], 
                    beta_lr[wsc_idx], tg_hsp[wsc_idx]
                )
                hsp_list[wsc_idx, epoch - 1] = hsp_val[wsc_idx]
                beta_list[wsc_idx, epoch - 1] = beta_val[wsc_idx]
                layer_reg = torch.norm(temp_w, 1) * beta_val[wsc_idx]
                wsc_idx += 1
            else:
                layer_reg = torch.norm(temp_w, 1) * l1_param
            l1_val += torch.norm(temp_w, 1)
            
            if l1_reg is None:
                l1_reg = layer_reg
            else:
                l1_reg = l1_reg + layer_reg
            layer_idx += 1
        
    return l1_reg, l1_val

In [24]:
def train(model, epoch, train_loader, optimizer, criterion,
          hsp_val, beta_val, hsp_list, beta_list, tg_hsp):    
    model.train()
    train_loss = 0
    total = 0
    train_acc = 0
    correct = 0
    
    for batch_idx, (input, target) in enumerate(train_loader):
        optimizer.zero_grad()
        input, target = input.to(DEVICE), target.to(DEVICE)
        output = model(input)
        loss = criterion(output, target)
        l1_term, l1_val = l1_penalty(
            model, epoch, hsp_val, beta_val, hsp_list, beta_list, tg_hsp)
        cost = loss + l1_term
        train_loss += loss
        cost.backward()
        optimizer.step()
        total += output.size(0)
        pred = (output.data > 0.5).float()
        correct += (pred == target).sum().item()

    train_loss /= total
    train_acc = 100 * correct / total
    return train_loss, train_acc, l1_val

In [25]:
def test(model, epoch, test_loader, criterion):
    model.eval()
    test_loss = 0
    test_acc = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for input, target in test_loader:
            input, target = input.to(DEVICE), target.to(DEVICE)
            output = model(input)
            loss = criterion(output, target)
            test_loss += loss.item()
            pred = (output.data > 0.5).float()
            total += output.size(0)
            correct += (pred == target).sum().item()
            
    test_loss /= total
    test_acc = 100 * correct / total
    return test_loss, test_acc

In [26]:
def plot_learning_curves(save_dir, epoch, 
                         plot_train_loss, plot_test_loss, 
                         plot_train_acc, plot_test_acc, 
                         plot_lr_list, plot_l1_list,
                         plot_hsp_list, plot_beta_list, tg_hsp):
    
    sns.set(style="darkgrid", font_scale=1.5)
    fig, ax = plt.subplots(3, 2, figsize=(14, 11))
    
    last_epoch = epoch

    ax = ax.flat
    lw = 2.5
    ax[0].plot(plot_train_loss[:last_epoch], label='train loss', lw=lw)
    ax[0].plot(plot_test_loss[:last_epoch], label='test loss', lw=lw)
    ax[0].set_title("Loss plot")
    ax[0].legend()
    
    ax[1].plot(plot_train_acc[:last_epoch], label='train acc', lw=lw)
    ax[1].plot(plot_test_acc[:last_epoch], label='test acc', lw=lw)
    ax[1].legend()
    ax[1].set_title("Accuracy plot ($acc$={:.2f}%)"
                       .format(plot_test_acc[-1]))
    
    ax[2].plot(plot_lr_list[:last_epoch], label='learning rate', lw=lw)
    ax[2].legend()
    ax[2].set_title("Learning rate plot")
    
    ax[3].plot(plot_l1_list[:last_epoch], label='learning rate', lw=lw)
    ax[3].legend()
    ax[3].set_title("L1 term plot")
    
    plot_hsp_list = np.array(plot_hsp_list).T
    plot_beta_list = np.array(plot_beta_list).T
    
    for idx, n_layer in enumerate(indices):
        if wsc_flag[idx] == 0:
            continue;
        ax[4].plot(plot_hsp_list[idx], label='layer{}'.format(n_layer), lw=lw)
        ax[5].plot(plot_beta_list[idx], 
                   label='layer{}'.format(n_layer), lw=lw)
        ax[4].legend(); ax[5].legend()
        ax[4].set_title("HSP plot [{:.3f}/{:.3f}]".format(plot_hsp_list[0, -1], tg_hsp[0]))
        ax[5].set_title("Beta plot")

    fig.tight_layout()
    fig.savefig("{}/Learning_curves.png".format(save_dir))
    plt.close(fig)

In [27]:
def run_fold(n_outer_cv=0, output_folder=output_folder):
    
    # Outer CV
    n_outer_cv = n_outer_cv
    output_folder = output_folder
    
    print("===================================== Outer Fold [{}/{}] =====================================".
          format(n_outer_cv + 1, outer_n_splits))
    outer_save_dir = "{}/Outer_fold_{}".format(output_folder, n_outer_cv+1)
    os.makedirs(outer_save_dir, exist_ok=True)
    outer_train_idx = train_folds_idx[n_outer_cv]
    outer_test_idx = test_folds_idx[n_outer_cv]

    # Assign train-valid dataset
    print("======================================== Test Phase =========================================")
    X_train, y_train = X[outer_train_idx], y[outer_train_idx]
    X_test, y_test = X[outer_test_idx], y[outer_test_idx]

    outer_train_dataset = train_dataset(X_train, y_train)
    outer_test_dataset = test_dataset(X_test, y_test)
    outer_train_loader = DataLoader(outer_train_dataset, batch_size=batch_size, 
                                    shuffle=True, num_workers=num_workers, drop_last=True)
    outer_test_loader = DataLoader(outer_test_dataset, batch_size=batch_size, 
                                    shuffle=True, num_workers=num_workers, drop_last=True)
    # Assign model 
    model = DNN().to(DEVICE)
    optimizer = get_optimizer(model, optimizer_name)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode=mode, patience=patience, 
                                               factor=factor, min_lr=min_lr)
    criterion = nn.BCEWithLogitsLoss(reduction='mean')
    
    # list to save learning parameters
    outer_train_loss = []
    outer_test_loss = []
    outer_train_acc = []
    outer_test_acc = []
    outer_lr_list = []
    outer_hsp_list = []
    outer_beta_list = []
    outer_l1_list = []
    
    hsp_val, beta_val, hsp_list, beta_list = init_hsp()

    for epoch in range(1, epochs+1):
        train_loss, train_acc, l1_val = train(
            model, epoch, outer_train_loader, optimizer, criterion, 
            hsp_val, beta_val, hsp_list, beta_list, tg_hsp)
        test_loss, test_acc = test(
            model, epoch, outer_test_loader, criterion)
        
        scheduler.step(hsp_val[0])
        lr = optimizer.param_groups[0]['lr']
        
        outer_train_loss.append(train_loss)
        outer_test_loss.append(test_loss)
        outer_train_acc.append(train_acc)
        outer_test_acc.append(test_acc)
        outer_lr_list.append(lr)
        outer_hsp_list.append(list(hsp_val))
        outer_beta_list.append(list(beta_val))
        outer_l1_list.append(l1_val)
        
        if epoch % 1 == 0:
            print("Epoch: [{:d}/{:d}], Train acc: {:.2f}, Test acc: {:.2f}".
                  format(epoch, epochs, train_acc, test_acc), end=", ")
            print("Train loss: {:.4f}, Test loss: {:.4f}".
                  format(train_loss, test_loss))
            for i in range(len(n_hidden)):
                if wsc_flag[i] != 0:
                    print("Layer {:d}: [{:.4f}/{:.4f}]".
                          format(i+1, hsp_val[i], tg_hsp[i]), end=" ")
            print("Current learning rate: {:.2e}".format(Decimal(str(lr))))

            plot_learning_curves(
                outer_save_dir, epochs, 
                outer_train_loss, outer_test_loss, 
                outer_train_acc, outer_test_acc, 
                outer_lr_list, outer_l1_list,
                outer_hsp_list, outer_beta_list, tg_hsp
            )
            if n_outer_cv == 0:
                torch.save(model.state_dict(), outer_save_dir + "/model.pt")
            
    print("Outer Fold [{}/{}], selected HSP: {}".
          format(n_outer_cv+1, outer_n_splits, tg_hsp))
    print("Test accuracy: {:.2f}\n".format(test_acc))
    return train_acc, test_acc

In [ ]:
hsp_loop_results = []

for tg_hsp in hsp_cand_list:
    hsp_output_folder = output_folder + "/" + str(tg_hsp)
    print(hsp_output_folder)
    
    folds_acc = []
    
    for i in range(outer_n_splits):
        fold_train_acc, fold_test_acc = run_fold(
            n_outer_cv=i, output_folder=hsp_output_folder
        )
        folds_acc.append([fold_train_acc, fold_test_acc])
        
    acc_df = pd.DataFrame(folds_acc, columns=["train", "test"])
    acc_df.to_csv("{}/acc_results.csv".format(hsp_output_folder))
    print("Test accuracy in {}: {:.3f}".format(tg_hsp, acc_df["test"].mean()))

    # draw barplot
    sns.set(style="whitegrid", font_scale=1.75)
    fig, ax = plt.subplots(figsize=(14, 6))
    sns.barplot(y="test", x=acc_df.index+1, data=acc_df, ax=ax, palette="deep")
    ax.set_xlabel("Fold")
    ax.set_ylabel("Accuracy")
    ax.set_title("Leave-one-site-out CV results (avg test acc: {:.3f})"
                 .format(acc_df["test"].mean()))
    fig.savefig("{}/bar_graph.png".format(hsp_output_folder))
    
    hsp_loop_results.append([tg_hsp, folds_acc])

/users/hjw/data/ABCD/DNN/210404_122020/[0.95, 0.8]
===================================== Outer Fold [1/5] =====================================
======================================== Test Phase =========================================
Epoch: [1/500], Train acc: 50.59, Test acc: 47.66, Train loss: 0.0092, Test loss: 0.0065
Layer 1: [0.2074/0.9500] Layer 2: [0.2048/0.8000] Current learning rate: 1.00e-3
Epoch: [2/500], Train acc: 52.54, Test acc: 49.22, Train loss: 0.0085, Test loss: 0.0066
Layer 1: [0.2332/0.9500] Layer 2: [0.2184/0.8000] Current learning rate: 1.00e-3
Epoch: [3/500], Train acc: 53.42, Test acc: 52.34, Train loss: 0.0082, Test loss: 0.0060
Layer 1: [0.2875/0.9500] Layer 2: [0.2472/0.8000] Current learning rate: 1.00e-3
Epoch: [4/500], Train acc: 58.30, Test acc: 54.69, Train loss: 0.0073, Test loss: 0.0053
Layer 1: [0.3717/0.9500] Layer 2: [0.2934/0.8000] Current learning rate: 1.00e-3
Epoch: [5/500], Train acc: 58.50, Test acc: 58.98, Train loss: 0.0074, Test loss: 

In [ ]:
results_df = pd.DataFrame(hsp_loop_results, columns=["hsp", "result"])
results_df.head()

In [ ]:
acc_df = pd.DataFrame(folds_acc, columns=["train", "test"])
cv_df = acc_df.reset_index()
cv_df = cv_df.rename(columns={"index": "fold"})
cv_df["fold"] = cv_df["fold"] + 1
cv_df

In [ ]:
sns.set_context("talk"); sns.set_style("white"); sns.set_palette("Pastel1")
fig, ax = plt.subplots()
sns.barplot(x="fold", y="test", data=cv_df, ax=ax)
ax.set_title("5-Fold CV Results from DNN (Acc: {:.2f}%)".format(cv_df.test.mean()), pad=12)
ax.set_ylabel("Accuracy (%)")
ax.set_xlabel("Fold")